In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from source.utils.config_manager import ConfigManager
from torchvision import models
import torchvision
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from source.utils.label_mapper import LabelMapper
from source.utils import save_to_kaggle

#hyperparameters tuning
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from source.hyperparams_tuning import HyperparameteresTunner

# transformers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loaders
from source.dataloaders.cnn_data_loaders_factory import CNNDataLoaderFactory


# networks
from source.custom_cnn.pytorch_ray_tutorial_cnn import Net
from source.custom_cnn.dropout_batch_norm_cnn import DBN_cnn
from source.pre_trained.vgg16 import PretrainedVGG16_cnn
from source.pre_trained.resnet_cnn import PretrainedRESNET_cnn
from source.pre_trained.efficient_net import PretrainedEff_cnn



In [2]:
dataset_name = 'cifar10'
dataset_path = ConfigManager.get_dataset_path(dataset_name)
loader = CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_transformer_kaggle(), TestTransformersFactory.get_transformer_efficient())
test_loader = loader.get_test_loader(64)

In [3]:
model1 = PretrainedEff_cnn()
model1.load_state_dict(torch.load("C:\\Users\\gosia\\Desktop\\studia\\mgr_sem_1\\DL\\cifar10-computer_vision\\source\\pretrained_results\\efficient.pth",map_location=torch.device('cpu')))
model1.eval()

PretrainedEff_cnn(
  (model): EfficientNet(
    (features): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_a

In [4]:
device = 'cuda'
outputs = model1.predict(test_loader, device,LabelMapper.get_cifar10_mapper()) 

  0%|          | 3/4688 [01:37<42:25:20, 32.60s/it]


KeyboardInterrupt: 

In [ ]:
save_to_kaggle(300000,outputs, "efficient")